In [1]:
import numpy as np

In [21]:
def rbf(nin, nhidden, nout, actfn, outfunc , prior = None, beta = None):
    """Create a net structure of RBF, we use dictionary.
       nin, number of inputs
       nhidden, number of hidden units
       nout, number of outputs
       nwts, total number of weights and biases
       actfn, activation function for hidden unit
       outfn, output error function
       prior, inverse variance value of a zero-mean Gaussian
       beta, inverse noise variance
    """
    rbf_net = {}
    rbf_net['nin'] = nin
    rbf_net['nhidden'] = nhidden
    rbf_net['nout'] = nout
    rbf_net['actfn'] = actfn
    rbf_net['outfunc'] = outfunc
    rbf_net['nwts' ] = nin*nhidden + (nhidden + 1)*nout + nhidden
    rbf_net['prior'] = prior
    rbf_net['beta'] = beta
    rbf_net['wi'] = np.ones(nhidden)
    w = np.random.randn(1, net['nwts'])
    net = rbfunpak(w, rbf_net)
    return net
    

In [55]:
def rbfunpak(w , net):
    nin = net['nin']
    nhidden = net['nhidden']
    nout = net['nout']
    mark1 = nin*nhidden
    net['c'] = np.reshape(w[0,0:mark1],(nhidden,nin))
    mark2 = mark1 + nhidden
    net['wi'] = np.reshape(w[0, mark1+1:mark2],(1,nhidden))
    mark3 = mark2 + nhidden*nout
    net['w2'] = np.reshape(w[0,mark2+1:mark3],(nhidden, nout))
    mark4 = mark3 + nout
    net['b2'] = np.reshape(w[0,mark3+1:mark4],(1,nout))
    

In [8]:
import sys

def rbfsetfw(net, scale):
    real_max = sys.float_info.max
    cdist =  np.linalg.norm(net['c'] - net['c'])
    if cdist > 0:
        cdist = cdist + real_max* np.eye(net['nhidden'])
        widths = scale*np.mean(min(cdist))
    else:
        widths = max(max(cdist))
    
    net['wi'] = widths * np.ones(np.shape(net['wi']))
    
        

In [5]:
def rbffwd(x, net):
    ndata = np.shape(x)
    n2 = np.linalg.norm(x - net['c'])
    wi2 = np.ones(3,1)*(2*net['wi'])
    z = np.exp(-(n2/wi2))
    a = z*net['w2'] + np.ones(ndata,1)*net['b2']
    return a, z, n2

In [86]:
def gmmactiv(x, mix):
    ndata = np.shape(x)[0]
    a = np.zeros(ndata, mix['ncentres'])
    
    n2 = np.linalg.norm(x - mix['centres']) # dist2
    n2 = np.array(n2)
    wi2 = np.ones((ndata,1))*  (2* mix['covars'])
    nr, nc = np.shape(wi2)
    for i in range(0, nr):
        for j in range(0, nc):
            if wi2[i,j] == 0:
                wi2[i,j] = eps
    
    normal = (np.pi * wi2)**(mix['nin']/2)
    a = np.exp((-n2/wi2)/normal)
    return a
    
def gmmpost(x,mix):
    
    
    ndata = np.shape(x)[0]
    a = gmmactive(mix, x)
    old_post = np.ones((ndata,1))*mix['priors']*a
    s = np.sum(post, axis = 1) # s = sum(post,2)
    post = old_post/(s*np.ones((1,mix['ncentres'])))
    post = np.array(post)
                     
                     
    return [post, a]       

In [88]:
def gmmem(x, mix, options = None):
    v = np.array([], dtype = 'f')
    for n in range(0,100):
        [post, act] = gmmpost(x,mix)
        
        #adjust the new estimate for the parameters
        new_pr = np.sum(post, axis = 0)
        new_c = post*x
        
        #itetrate
        mix['priors'] = new_pr/ndata
        mix['centres'] = new_c/(new_pr * np.ones((1,mix['nin'])))
        if mix['covar_type'] == 'spherical':
            n2 = np.linalg.norm(x - mix['centres'])
            
            for i in range(0, mix['ncentres']):
                v_i = (post[:,i])*(n2[:,i])
                v.append(v_j)
            mix['covars'] = np.array([(var/new_pr)/mix['nin'] for var in v])
            
            return mix   

In [12]:
def gmm(dim, ncentres, covar_type):
    
    mix = {}
    mix['type'] = 'gmm'
    mix['nin'] = dim
    mix['ncentres'] = ncentres
    mix['covar_type'] = covar_type  #spherical
    mix['priors'] = np.ones((1,ncentres))/(ncentres)
    mix['centres'] = np.random.randn(ncentres, nin)
    mix['covars'] = np.ones((1, ncentres))
    mix['nwts'] = ncentres + ncentres*nin + ncentres
    return mix
    

In [16]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x)) #axis = 0

def dmm(dim, ncentres, dist_type, nvalues, a = None, b = 1):
    mix = {}
    mix['type'] = 'dmm'
    mix['input_dim'] = dim
    mix['ncentres'] = ncentres
    mix['dist_type'] = dist_type
    mix['priors'] = np.ones((1, ncentres)) / ncentres
    if dist_type == 'bernoulli':
        mix['nvalues'] = 1
        mix['nin'] = dim
        mix['means'] = np.random.rand(ncentres, dim)
        if a < 1:
            print('a gives a singular prior')
        else:
            mix['a'] = a
        if b < 1:
            print('b gives a singular prior')
        else:
            mix['b'] = b
            
    elif dist_type == 'multinomial':
        mix['nvalues'] = nvalues
        mix['nin'] = np.sum(nvaluse)
        mix['mean'] = np.zeros((ncentres, dim))
        k = 0
        a = np.shape(mix['nvalues'][1])
        for i in range(0, a):
            mix['means'][:,k+1:k+mix['nvaluse'][i]] = softmax(np.random.randn(ncentres,nvaluse[i]))
            k = mix['nvalues'][i]
            if a < 1:
                print('a gives a singular prior')
            else :
                mix['a'] = a
    else:
        print('unknown distribution.')
        
        
    

    

In [18]:
def rbfprior(rbfunc, nin, nhidden, nout, aw2, ab2):
    nwts_layer2 = nout + (nhidden *nout)
    if rbfunc == 'gaussian':
        nwts_layer1 = nin*nhidden + nhidden
    else:
        print('Undefined activation function')
    nwts = nwts_layer1 + nwts_layer2
    mask = [np.zeros((nwts_layer1, 1)), np.ones((nwts_layer2, 1))]
    indx = np.zeros((nwts, 2))
    mark2 = nwts_layer1 + (nhidden * nout)
    indx[nwts_layer1 + 1:mark2, 1] = np.ones((nhidden * nout, 1))
    indx[mark2 + 1:nwts, 2] = np.ones((nout, 1))
    prior ={'index':indx, 'alpha': [aw2, ab2]}
    return mask, prior

   
    


In [15]:
def dmmactiv(mix,x):
    ndata = np.shape(x)[0]
    a = np.zeros((ndata, mix['ncentres']))
    e = np.ones((ndata,1))
    if mix['dist_type'] == 'bernoulli':
        for m in range(mix['ncentres']):
            a[:,m] =  np.prod(((e*(mix['means'][m,:]))**x)*
                 ((e*(1-mix['means'][m,:]))**(1-x)), 2)
    elif mix['dist_type'] == 'multinomial':
        for m in range(mix['ncentres']):
            a[:,m] = np.prod(((e*(1-mix['means'][m,:]))**(1-x)), 2)
    else:
        print('unknown distribution type.')
           

In [9]:
def dmmpost(mix,x):
    a = dmmactiv(mix,x)
    ndata = np.shape(x)[0]
    post = (np.ones(ndata)[0]*mix['priors']*a)
    s = np.sum(post, axis = 0)
    post = post/(s*np.ones((1,mix['ncenres'])))
    return post
    
    
